<a href="https://colab.research.google.com/github/srilamaiti/srilamaiti.github.io/blob/main/ai_powered_financial_market_research/financial_research_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install new libraries

In [1]:
!pip install groq
!pip install yfinance
!pip install agno
!pip install duckduckgo-search
!pip install google-search-results
!pip install newspaper4k
!pip install lxml_html_clean
!pip install -U sqlalchemy 'psycopg[binary]'
!pip install pgvector
!pip install pypdf
!pip install udocker
!pip install sentence-transformers
!pip install python-dotenv
!pip uninstall openai -y
!pip install openai
!pip install gradio
!pip install langgraph
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.0/700.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=15d4f09a2815e46fc90b7cb522a89b914b89c3e922cb3175e952d01a4a0a6f5c
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Importing libraries

In [2]:
import linecache
import os
import glob
import uuid
from transformers import pipeline
import openai
from openai import OpenAI
from dotenv import load_dotenv
import langchain
from requests.exceptions import HTTPError
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langgraph.graph import StateGraph
from langgraph.graph import START
from langgraph.graph import END
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langgraph.graph import MessagesState
from langchain.schema import AIMessage
from langchain.schema import HumanMessage
from langchain.schema import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain_community.tools.file_management import ReadFileTool
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain.utilities import SerpAPIWrapper
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import time
from datetime import datetime
import pprint
import gradio as gr
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


In [3]:
from IPython import get_ipython
from IPython.display import Markdown
from IPython.display import display
from IPython.display import Image

In [4]:
def pretty_print(text):
    display(Markdown(text))

# API key setup

In [5]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env
!cat /content/drive/MyDrive/groq_api_key.txt >> .env
!cat /content/drive/MyDrive/serpa_api_key.txt >> .env

In [6]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERPA_API_KEY = os.getenv("SERPA_API_KEY")

# Experiments

In [7]:
# Initialize the LLM
llm = OpenAI(temperature=0)

# Current date and time for the report
current_date = datetime.now().strftime("%B %d, %Y")
current_time = datetime.now().strftime("%H:%M:%S")

<ipython-input-7-8a7625e233a2>:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [8]:
#Initialize the web search tool
search = SerpAPIWrapper(serpapi_api_key=SERPA_API_KEY)
search_system_message = """Use this tool to perform Google-based web searches for
                           financial topics, prioritizing authoritative sources,
                           recent publications, and expert opinions. Search for
                           5 sources per query."""
search_tool = Tool(
    name="Financial Web Search",
    func=search.run,
    description=search_system_message
)

# Web Search Agent System Message
web_search_system_message = """You are an elite finance specialist with expertise
                               in sourcing authoritative financial market data,
                               particularly in real estate economics, mortgage,
                               and property information.
                               Your role is to identify credible sources for
                               current financial market analysis.

                               **Task**: Perform web searches to find 5 authoritative
                               sources for the user query. Follow these guidelines:
                              1. Use the Financial Web Search tool to conduct Google-based searches.
                              2. Prioritize recent publications (2024-2025)
                              from reputable organizations (e.g., NAR, Freddie Mac, Forbes,
                              J.P. Morgan, Bankrate).
                              4. Focus on expert opinions, industry reports, and data-driven articles.
                              5. Ensure sources are relevant to the query and provide actionable insights.
                              6. Output a list of sources with titles, URLs, and
                              a brief description of their contributions.

                              **Output Format**:
                              # Web Search Results
                              - Source 1: [Title, URL]
                              - Source 2: [Title, URL]
                              - Source 3: [Title, URL]
                              - Source 4: [Title, URL]
                              - Source 5: [Title, URL]
                           """

# Research Agent System Message
research_system_message = """You are an expert finance research analyst specializing
                             in real estate economics, mortgage, and property information.
                             Your role is to conduct deep investigative research
                             for financial market analysis. Your expertise includes:

                             - Fact-checking and source verification
                             - Extracting critical data and trends

                            **Task**: Perform the Research Phase using the provided
                            web search results. Follow these guidelines:
                            1. Analyze the 5 sources to extract relevant data, statistics, and expert opinions.
                            2. Verify facts by cross-referencing across sources.
                            3. Note emerging trends, conflicting perspectives, and critical statistics.
                            4. Output a structured research summary with:
                            - List of sources with contributions
                            - Key data points (statistics, trends, quotes)
                            - Experts' perspectives
                            - Verified facts and any discrepancies

                           **Output Format**:
                           # Research Summary
                           ## Sources
                           - Source 1: [Title, URL]
                           - Source 2: [Title, URL]
                           - Source 3: [Title, URL]
                           - Source 4: [Title, URL]
                           - Source 5: [Title, URL]
                           ## Key Data Points
                           - Statistic/Trend 1
                           - Statistic/Trend 2
                           - Statistic/Trend 3
                           ## Market Leaders' Perspectives
                           - Perspective 1
                           - Perspective 2
                           - Perspective 3
                          ## Expertts' Perspectives
                          - Expert opinion 1: Perspective
                          - Expert opinion 2: Perspective
                          - Expert opinion 3: Perspective
                          ## Verified Facts
                          - Fact 1
                          - Fact 2
                         ## Discrepancies
                          - Discrepancy 1 (if any)
                        """

# Summary Agent System Message
summary_system_message = """
You are an expert financial analyst specializing in summarizing complex real estate data.
Your role is to analyze research data and produce a concise executive summary. Your expertise includes:

- Extracting and verifying critical information
- Identifying emerging patterns and trends
- Evaluating conflicting viewpoints
- Simplifying complex topics
- Maintaining objectivity

**Task**: Perform the Analysis Phase using the research summary provided. Follow these guidelines:
1. Extract key findings from the research data.
2. Cross-reference facts to ensure accuracy.
3. Identify patterns, trends, and future implications.
4. Evaluate conflicting viewpoints for balance.
5. Produce a concise one pager executive summary highlighting the most critical insights.

**Output Format**:
# Executive Summary
[Concise overview of key findings and significance, one page]
"""

# Write-up Agent System Message
writeup_system_message = """
You are an efficient financial report writer specializing in real estate economics.
Your role is to create a comprehensive financial report based on research and summary data. Your expertise includes:

- Crafting engaging and objective narratives
- Structuring content in a financial report format
- Incorporating data, quotes, and expert insights
- Explaining complex concepts clearly
- Maintaining balance and ethical reporting

**Task**: Perform the Writing Phase using the research summary and executive summary. Follow these guidelines:
1. Craft an attention-grabbing headline.
2. Structure the report in the specified Financial Report style (Headline, Executive Summary,
Background & Context, Key Findings, Impact Analysis, Future Outlook, Sources, Methodology, Expert Insights).
3. Ensure narrative flow and readability.
4. Include relevant quotes, statistics, and context.
5. Maintain objectivity and balance.
6. Format the response as follows:

**Output Format**:
# Headline
[Attention-grabbing headline]

## Executive Summary
[Insert executive summary from Summary Agent]

## Background & Context
[Historical context and current landscape]

## Key Findings
[Main discoveries, expert insights, statistics]

## Impact Analysis
[Current implications, stakeholder perspectives, industry effects]

## Future Outlook
[Emerging trends, expert predictions, challenges, opportunities]

## Sources
[List of primary sources with contributions]

## Methodology
[Research methodology overview]

## Expert Insights
[Notable quotes and contrasting viewpoints]

--------------------------------------------
Research conducted by FA AI Financial Agent
Published: {current_date}
Last Updated: {current_time}
"""

# Initialize Agents
web_search_agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    system_message=web_search_system_message
)
research_prompt = PromptTemplate(
    input_variables=["web_search_response"],
    template="Conduct research based on these web search results:\n{web_search_response}\n\n{research_system_message}"
)

summary_prompt = PromptTemplate(
    input_variables=["research_response"],
    template="Create an executive summary based on this research summary:\n{research_response}\n\n{summary_system_message}"
)

writeup_filled = writeup_system_message.format(current_date=current_date, current_time=current_time)

writeup_prompt = PromptTemplate(
    input_variables=["research_response", "summary_response"],
    template="Create a financial report based on this research summary:\n{research_response}\n\nAnd this executive summary:\n{summary_response}\n\n" + writeup_filled
)
'''
# Query
query = f"How is the current housing market condition in the USA as of {current_date}?"

# Step 1: Web Search Agent
web_search_response = web_search_agent.run(query)

# === Step 2: Research Agent (LLMChain) ===
research_prompt = PromptTemplate(
    input_variables=["web_search_response"],
    template="Conduct research based on these web search results:\n{web_search_response}\n\n" + research_system_message
)
research_agent = LLMChain(llm=llm, prompt=research_prompt)
research_response = research_agent.run(web_search_response=web_search_response)

# === Step 3: Summary Agent (LLMChain) ===
summary_prompt = PromptTemplate(
    input_variables=["research_response"],
    template="Create an executive summary based on this research summary:\n{research_response}\n\n" + summary_system_message
)

summary_agent = LLMChain(llm=llm, prompt=summary_prompt)
summary_response = summary_agent.run(research_response=research_response)

# === Step 4: Write-up Agent (LLMChain) ===
writeup_prompt = PromptTemplate(
    input_variables=["research_response", "summary_response"],
    template="Create a financial report based on this research summary:\n{research_response}\n\nAnd this executive summary:\n{summary_response}\n\n" + writeup_system_message
)
writeup_agent = LLMChain(llm=llm, prompt=writeup_prompt)
writeup_response = writeup_agent.run({
    "research_response": research_response,
    "summary_response": summary_response,
    "current_date": current_date,
    "current_time": current_time
})

# === Output ===
print(writeup_response)

# Optional: Save to markdown file
with open("financial_report.md", "w", encoding="utf-8") as f:
    f.write(writeup_response)
'''

<ipython-input-8-4c79ddcff0ee>:166: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  web_search_agent = initialize_agent(
<ipython-input-8-4c79ddcff0ee>:194: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  web_search_response = web_search_agent.run(query)




> Entering new AgentExecutor chain...
 I should use Financial Web Search to find authoritative sources on the current housing market condition in the USA.
Action: Financial Web Search
Action Input: "current housing market condition USA May 07, 2025"
Observation: [' entity_type: related_questions.', "Home prices are declining in some markets—and rising in others. Here's where experts predict the housing market is headed in 2025.", "There's still plenty of uncertainty in the air, though: Rising prices and slowing construction could cause some trouble for buyers in 2025. And ...", 'The US housing market is likely to remain largely frozen through 2025. Some growth is still expected, but at a very subdued pace of 3% or less.', '2025-2029 Five-Year Housing Market Predictions: The next five years will likely usher in slower increases in both home prices and rents.', 'And yes, selling a home in 2025 will come with obstacles—like high interest rates and home values pricing out a lot of would-

<ipython-input-8-4c79ddcff0ee>:201: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  research_agent = LLMChain(llm=llm, prompt=research_prompt)


--------------------------------------------

# US Housing Market Forecast 2025: A Comprehensive Analysis of Key Trends and Predictions

## Executive Summary
The US housing market is a dynamic and ever-changing industry, influenced by a multitude of factors. Through extensive research and analysis of various sources, including Norada Real Estate Investments, Forbes, National Association of Realtors, Mashvisor, and Redfin, we have identified key trends and patterns that will shape the market in the coming years. Our findings reveal a projected growth in the housing market, driven by strong demand and low mortgage rates. However, there are also conflicting viewpoints on the future of the market, highlighting the importance of evaluating different perspectives. Additionally, emerging trends such as the increasing popularity of real estate investing and a shift towards more affordable and suburban housing options will impact the market in the near future.

## Background & Context
The US ho

In [9]:
def generate_report(query):
    # Current date and time
    current_date = datetime.now().strftime("%B %d, %Y")
    current_time = datetime.now().strftime("%H:%M:%S")

    # Step 1: Web Search
    web_search_response = web_search_agent.run(query)

    # Step 2: Research
    research_prompt = PromptTemplate(
        input_variables=["web_search_response"],
        template="Conduct research based on these web search results:\n{web_search_response}\n\n" + research_system_message
    )
    research_agent = LLMChain(llm=llm, prompt=research_prompt)
    research_response = research_agent.run(web_search_response=web_search_response)

    # Step 3: Summary
    summary_prompt = PromptTemplate(
        input_variables=["research_response"],
        template="Create an executive summary based on this research summary:\n{research_response}\n\n" + summary_system_message
    )
    summary_agent = LLMChain(llm=llm, prompt=summary_prompt)
    summary_response = summary_agent.run(research_response=research_response)

    # Step 4: Write-up
    writeup_filled = writeup_system_message.format(current_date=current_date, current_time=current_time)
    writeup_prompt = PromptTemplate(
        input_variables=["research_response", "summary_response"],
        template="Create a financial report based on this research summary:\n{research_response}\n\nAnd this executive summary:\n{summary_response}\n\n" + writeup_filled
    )
    writeup_agent = LLMChain(llm=llm, prompt=writeup_prompt)
    writeup_response = writeup_agent.run({
        "research_response": research_response,
        "summary_response": summary_response
    })

    return writeup_response

In [10]:
# === Gradio UI ===
iface = gr.Interface(
    fn=generate_report,
    inputs=gr.Textbox(label="Enter your financial query", placeholder="e.g., How is the US housing market in 2025?"),
    outputs=gr.Textbox(label="Generated Financial Report", lines=25),
    title="Financial Market Report Generator",
    description="This tool performs multi-step financial analysis and generates a comprehensive report using live web search."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f95b7cc0cac18f2b3e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
